In [22]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.spatial import distance 
from sklearn.preprocessing import StandardScaler

# Loads each whole scenario as a sheet

In [5]:
start_folder = 'AllCSVs'
csvs_scen = glob.glob(r'{}\*.csv'.format(start_folder))
df_scen = []

for c in csvs_scen:
    df = pd.read_csv(c).drop(['type','messageID','spd_noise0','spd_noise1','spd_noise2' ,'pos_noise0' ,'pos_noise1' ,'pos_noise2', 'spd2', 'pos2'], axis=1)
    df_scen.append(df)
print(len(df_scen))
df_scen[0]

223


,time,sender,attackerType,pos0,pos1,spd0,spd1
0,10800.392785,13,0,3597.152086,5542.199221,-3.178366,38.798969
1,10800.437587,7,0,3609.398494,5446.803264,-3.534058,30.622234
2,10800.456150,31,0,3596.805861,5692.113417,1.282467,-29.864003
3,10800.582020,25,0,3597.770493,5764.800893,-0.020161,-34.568848
4,10800.670638,43,0,3822.892006,5266.806009,1.305362,-0.988730
...,...,...,...,...,...,...,...
954,10899.650338,79,0,6172.707296,5739.370670,14.429111,0.738493
955,10899.654916,217,0,3633.537216,5258.422180,-5.789623,37.114915
956,10899.670638,43,0,4492.896730,5263.131768,11.658342,-9.209199
957,10899.673155,205,0,3584.318133,5875.618176,-0.037875,39.077818


In [6]:
# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # select a row to find closest rows to
    given_row = row

    # Find the index of the given row
    row_index = df.index.get_loc(given_row.name)

    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[max(0, row_index-2000):row_index]

    if len(above_df) == 0:
        return pd.DataFrame()

    # select the A and B columns of the DataFrame
    df_ab = above_df[['time', 'pos0', 'pos1']]

    # calculate the Euclidean distance between the given row and all other rows based on A and B values
    distances = distance.cdist(df_ab, [given_row[['time', 'pos0', 'pos1']]], metric='euclidean')

    # get the indices of rows sorted by distance
    sorted_indices = distances.argsort(axis=0).flatten()

    # get the closest rows as a DataFrame
    closest_rows = df.iloc[sorted_indices]
    
    # Create a Boolean mask to select the rows that contain the value. Ensures that senders are not the same
    mask = closest_rows['sender'] == search_row.sender

    # Drop the rows that match the mask
    closest_rows.drop(closest_rows[mask].index, inplace=True)

    return closest_rows

In [7]:
df_closest = []
p = 0
for df in df_scen:
    df_copy = df.copy()
    for i in range(2, len(df_copy)): #len(df)
#         print(i)
#         display(search_row)
        search_row = df_copy.iloc[i] 
        closest_rows = find_closest_rows_above(df.copy(), search_row).reset_index()
        closest_car_nums = [0,1]
        feats = ['pos0','pos1','spd0','spd1']
#         display(closest_rows)
        for num in closest_car_nums:
            for f in feats:
                df_copy.loc[i, f'{f}Close{num}'] = closest_rows.loc[num, f]
    
    df_copy.to_csv(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\scenario_{}.csv'.format(p))  
    df_closest.append(df_copy)
    p+=1

In [8]:
display(df_closest[1])

,time,sender,attackerType,pos0,pos1,spd0,spd1,pos0Close0,pos1Close0,spd0Close0,spd1Close0,pos0Close1,pos1Close1,spd0Close1,spd1Close1
0,10800.034171,37,0,3606.595310,5591.689019,3.568682,-42.388476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10800.137475,67,1,3598.982868,5479.571773,-3.930110,34.067916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10800.186260,7,0,3587.643974,5848.923665,-0.032623,33.623910,3606.595310,5591.689019,3.568682,-42.388476,3598.982868,5479.571773,-3.930110,34.067916
3,10800.198101,19,0,3595.990733,5596.737221,-2.850521,34.797342,3606.595310,5591.689019,3.568682,-42.388476,3598.982868,5479.571773,-3.930110,34.067916
4,10800.443453,13,0,3589.278088,5923.118910,1.427552,33.529356,3587.643974,5848.923665,-0.032623,33.623910,3595.990733,5596.737221,-2.850521,34.797342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,10899.571009,73,0,4496.241972,5260.454798,12.764543,-10.216198,4470.265033,5276.478386,11.391257,-8.860582,4459.010477,5285.232636,10.862648,-8.241651
1137,10899.608602,205,0,3646.601126,5230.382731,5.077941,-32.256467,3637.904226,5230.427053,-5.269014,33.777508,3635.833657,5243.700628,-5.815339,37.279775
1138,10899.646511,115,0,6177.391286,5739.610400,15.297208,0.782922,6214.829707,6028.902079,14.175848,-5.118337,6200.557077,6034.039960,14.286154,-4.842704
1139,10899.765485,61,0,4481.710098,5267.652385,11.482956,-8.845861,4483.345992,5270.558280,12.991564,-10.008012,4496.241972,5260.454798,12.764543,-10.216198


In [9]:
start_folder = 'ClosestCarCSVsModel2'
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
csvs

['ClosestCarCSVsModel2\\scenario_0.csv',
 'ClosestCarCSVsModel2\\scenario_1.csv',
 'ClosestCarCSVsModel2\\scenario_10.csv',
 'ClosestCarCSVsModel2\\scenario_100.csv',
 'ClosestCarCSVsModel2\\scenario_101.csv',
 'ClosestCarCSVsModel2\\scenario_102.csv',
 'ClosestCarCSVsModel2\\scenario_103.csv',
 'ClosestCarCSVsModel2\\scenario_104.csv',
 'ClosestCarCSVsModel2\\scenario_105.csv',
 'ClosestCarCSVsModel2\\scenario_106.csv',
 'ClosestCarCSVsModel2\\scenario_107.csv',
 'ClosestCarCSVsModel2\\scenario_108.csv',
 'ClosestCarCSVsModel2\\scenario_109.csv',
 'ClosestCarCSVsModel2\\scenario_11.csv',
 'ClosestCarCSVsModel2\\scenario_110.csv',
 'ClosestCarCSVsModel2\\scenario_111.csv',
 'ClosestCarCSVsModel2\\scenario_112.csv',
 'ClosestCarCSVsModel2\\scenario_113.csv',
 'ClosestCarCSVsModel2\\scenario_114.csv',
 'ClosestCarCSVsModel2\\scenario_115.csv',
 'ClosestCarCSVsModel2\\scenario_116.csv',
 'ClosestCarCSVsModel2\\scenario_117.csv',
 'ClosestCarCSVsModel2\\scenario_118.csv',
 'ClosestCarCSVsM

In [11]:
# Next Step **Chop out the first 2 rows of the df**
start_folder = 'ClosestCarCSVsModel2'
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
df_list = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
print(len(df_list))
# df_list[0]

47539


In [31]:
df_attacker = []
df_normal = []
for j in df_list:
    if j.attackerType.any() == 0:       
        df_temp = j.iloc[:,4:].fillna(0)
        scaler = StandardScaler()
        df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
        df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
        df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
        df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
        df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
        df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])
#         display(df_temp)
        
        j = np.asarray(df_temp).astype('float32')
        j = np.pad(j, ((0,100-j.shape[0]),(0,0)), 'constant')
        df_normal.append(j)
#         print(np.asarray(j).shape)
    elif j.attackerType.any() == 1:
        df_temp = j.iloc[:,4:].fillna(0)
        scaler = StandardScaler()
        df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
        df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
        df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
        df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
        df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
        df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])

        j = np.asarray(df_temp).astype('float32')
        j = np.pad(j, ((0,100-j.shape[0]),(0,0)), 'constant')
        df_attacker.append(j)

df_normal = np.asarray(df_normal)
df_attacker = np.asarray(df_attacker)

df_normal.shape

(37956, 100, 12)

In [36]:
df_attacker[60]

array([[ 1.32317   ,  1.3234301 ,  2.431628  , ..., -0.7241989 ,
         1.3652132 , -0.03662435],
       [ 1.2684344 ,  1.2686571 ,  1.8353267 , ...,  0.37670413,
        -0.4554882 ,  0.06419499],
       [ 1.1785216 ,  1.1786828 ,  1.3651093 , ...,  0.51782507,
        -0.19228329, -1.6964359 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [35]:
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_normal.npy', df_normal)
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_attacker.npy', df_attacker)


# ------------------------------------------Template Code-----------------------------------------

In [ ]:
import pandas as pd
from scipy.spatial import distance

# create a sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 4,6,7,8,9],
    'B': [2, 4, 6, 8, 7,3,7,3,8],
    'C': [3, 6, 9, 12, 3,2,9,6,4]
})

# select a row to find closest rows to
given_row = df.iloc[4]

# Find the index of the given row
row_index = df.index.get_loc(given_row.name)

# Slice the DataFrame to only consider rows above the given row
above_df = df.iloc[:row_index]

# if len(above_df) == 0:
#     return pd.DataFrame()


# select the A and B columns of the DataFrame
df_ab = above_df[['A', 'B']]

# calculate the Euclidean distance between the given row and all other rows based on A and B values
distances = distance.cdist(df_ab, [given_row[['A', 'B']]], metric='euclidean')

# get the indices of rows sorted by distance
sorted_indices = distances.argsort(axis=0).flatten()

# select the two closest rows (excluding the given row itself)
closest_indices = sorted_indices[0:3]

# get the closest rows as a DataFrame
closest_rows = df.iloc[closest_indices]

print(closest_rows)

In [ ]:
import pandas as pd
from scipy.spatial.distance import cdist

# Generate a sample DataFrame
df = pd.DataFrame({'A': [1, 1, 1, 4, 1],
                   'B': [6, 2, 2, 9, 2],
                   'C': [11, 3, 3, 14, 3]})

# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # Find the index of the given row
    row_index = df.index.get_loc(row.name)
    
    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[:row_index]
    
    if len(above_df) == 0:
        return pd.DataFrame()
    
    # Compute the distances between the given row and all rows above it in the DataFrame based on A and B values
    distances = cdist(above_df[['A', 'B']].values, row[['A', 'B']].values.reshape(1, -1))
    
    # Sort the distances in ascending order and retrieve the index of the closest row
    closest_index = distances.argmin(axis=0)[0]
    
    # Use the index to retrieve the closest row from the DataFrame
    closest_row = above_df.iloc[closest_index]
    
    # Find the index of the second closest row
    second_closest_index = (distances != distances[closest_index]).argmax(axis=0)[0]
    
    # Use the index to retrieve the second closest row from the DataFrame
    second_closest_row = above_df.iloc[second_closest_index]
    
    return pd.concat([closest_row, second_closest_row], axis=1).T

# Define a sample row to search for
search_row = df.iloc[2] 
# For i in range(0, len(df)): 
#     search_row = df.iloc[i]
# Append the closest_rows to a new dataframe and then glue the 2 dataframes together at the end




# Find the two closest rows above the search row based on A and B values
closest_rows = find_closest_rows_above(df, search_row)

print(closest_rows)

In [ ]:
df

In [ ]:
import pandas as pd

# create a DataFrame
df = pd.DataFrame({'pos1': [25, 30, 35],
                   'pos2': [25, 30, 35],
                   'pos3': [25, 30, 35]})

# create a new DataFrame with the new columns
closest_rows = pd.DataFrame({'pos1DIFF': [50000, 60000, 70000],
                         'pos2DIFF': [1, 2, 3]})

# concatenate the new DataFrame to the existing one
result = pd.concat([df, closest_rows], axis=1)

print(result)

In [ ]:
X = []
y = []
for i in df_normal:
    X.append(np.asarray(df_normal[0].iloc[0:10,2:]))
    y.append(np.asarray(df_normal[0].iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\X.npy'.format(start_folder), X)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\y.npy'.format(start_folder), y)